# Use append_audio()

In [1]:
# As imported module
from append_audio import append_session_audio

# Process all sessions
processed = append_session_audio(
    base_dir="../_data/audio",
    output_dir="../_data/audio/all_session_test",
    delete_source=False
)


Processing session_23
['audio_23_1.wav', 'audio_23_2.wav', 'audio_23_3.wav', 'audio_23_4.wav', 'audio_23_5.wav']
Created: ../_data/audio/all_session_test/session_23.wav

Processing session_25
['audio_25_1.wav', 'audio_25_2.wav', 'audio_25_3.wav', 'audio_25_4.wav', 'audio_25_5.wav', 'audio_25_6.wav', 'audio_25_7.wav', 'audio_25_8.wav', 'audio_25_9.wav', 'audio_25_10.wav', 'audio_25_11.wav', 'audio_25_12.wav', 'audio_25_13.wav', 'audio_25_14.wav', 'audio_25_15.wav', 'audio_25_16.wav', 'audio_25_17.wav', 'audio_25_18.wav', 'audio_25_19.wav', 'audio_25_20.wav', 'audio_25_21.wav', 'audio_25_22.wav', 'audio_25_23.wav', 'audio_25_24.wav', 'audio_25_25.wav', 'audio_25_26.wav', 'audio_25_27.wav', 'audio_25_28.wav', 'audio_25_29.wav', 'audio_25_30.wav', 'audio_25_31.wav', 'audio_25_32.wav', 'audio_25_33.wav', 'audio_25_34.wav', 'audio_25_35.wav', 'audio_25_36.wav', 'audio_25_37.wav', 'audio_25_38.wav', 'audio_25_39.wav', 'audio_25_40.wav', 'audio_25_41.wav', 'audio_25_42.wav', 'audio_25_43.wav'

---

# Removing Silence

In [2]:
!pip install webrtcvad

  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
  Created wheel for webrtcvad: filename=webrtcvad-2.0.10-cp311-cp311-macosx_11_0_arm64.whl size=26149 sha256=cab1d9de939e197b13956a2172ffd6a4f153f8a7db7300862ac8f94ed8a14c09
  Stored in directory: /Users/tillmann/Library/Caches/pip/wheels/94/65/3f/292d0b656be33d1c801831201c74b5f68f41a2ae465ff2ee2f
Successfully built webrtcvad


In [15]:
import webrtcvad
import wave

vad = webrtcvad.Vad(1)

def process_audio(audio_file,aggressiveness =1, frame_duration=30):
    vad = webrtcvad.Vad(aggressiveness)  # 0 to 3, while 3 is the most aggressive
    
    with wave.open(audio_file, 'rb') as wf:
        sample_rate = wf.getframerate()
        
        # Calculate frame size (must be 10, 20, or 30 ms for WebRTC VAD)
        if frame_duration not in [10, 20, 30]:
            frame_duration = 30
            
        frame_size = int(sample_rate * frame_duration / 1000)
        
        segments = []

        while True:
            frame = wf.readframes(frame_size)
            if not frame:
                break
                
            # Ensure we have enough samples for a complete frame
            if len(frame) < frame_size * 2:  # *2 because 16-bit samples
                break
                
            try:
                is_speech = vad.is_speech(frame, sample_rate)
                segments.append({
                    'start': wf.tell() / sample_rate - frame_duration / 1000,
                    'end': wf.tell() / sample_rate,
                    'is_speech': is_speech
                })
            except Exception as e:
                print(f"Error processing frame: {e}")
                continue

        return segments 

# Process the audio file
segments = process_audio("../_data/audio/all_sessions/session_23.wav")

# Print the segments
for segment in segments:
    print(segment)

{'start': 0.0, 'end': 0.03, 'is_speech': True}
{'start': 0.03, 'end': 0.06, 'is_speech': True}
{'start': 0.06, 'end': 0.09, 'is_speech': True}
{'start': 0.09, 'end': 0.12, 'is_speech': True}
{'start': 0.12, 'end': 0.15, 'is_speech': True}
{'start': 0.15, 'end': 0.18, 'is_speech': False}
{'start': 0.18, 'end': 0.21, 'is_speech': False}
{'start': 0.21, 'end': 0.24, 'is_speech': False}
{'start': 0.24000000000000002, 'end': 0.27, 'is_speech': False}
{'start': 0.27, 'end': 0.3, 'is_speech': False}
{'start': 0.30000000000000004, 'end': 0.33, 'is_speech': False}
{'start': 0.32999999999999996, 'end': 0.36, 'is_speech': False}
{'start': 0.36, 'end': 0.39, 'is_speech': False}
{'start': 0.39, 'end': 0.42, 'is_speech': False}
{'start': 0.42000000000000004, 'end': 0.45, 'is_speech': False}
{'start': 0.44999999999999996, 'end': 0.48, 'is_speech': False}
{'start': 0.48, 'end': 0.51, 'is_speech': False}
{'start': 0.51, 'end': 0.54, 'is_speech': False}
{'start': 0.5399999999999999, 'end': 0.57, 'is_spe

### Creating new file without speech.

In [25]:
# Create a new audio file with only the speech segments, using the process_audio function

def save_speech(audio_file, output_file, segments):
    with wave.open(audio_file, 'rb') as wf:
        sample_width = wf.getsampwidth()
        sample_rate = wf.getframerate()
        n_channels = wf.getnchannels()
        
        with wave.open(output_file, 'wb') as output_wf:
            output_wf.setnchannels(n_channels)
            output_wf.setsampwidth(sample_width)
            output_wf.setframerate(sample_rate)
            
            for segment in segments:
                if segment['is_speech']:
                    wf.setpos(int(segment['start'] * sample_rate))
                    output_wf.writeframes(wf.readframes(int((segment['end'] - segment['start']) * sample_rate)))

# Process the audio file

audio_file = "../_data/audio/all_sessions/session_18"

# Save the speech segments to a new audio file
save_speech(f'{audio_file}.wav', f'{audio_file}_speech.wav' , process_audio(f'{audio_file}.wav'))